In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from torchvision import datasets,transforms
from sklearn.model_selection import train_test_split

import pandas as pd

import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
%matplotlib inline

In [2]:
train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ]))
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ]))

In [3]:
train_loader = DataLoader(train_data,  batch_size=100, shuffle=True)

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(256, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.reshape(-1, 256)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = Net()
model = model.cuda()
torch.backends.cudnn.benchmark=True

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [6]:
for epoch in range(10):
    total_loss = 0
    for data, target in train_loader:
        #CNN filter에 들어가는값 [batch, filters, heigth, width]
        train_x, train_y = Variable(data.reshape(-1, 1, 28, 28).float()).cuda(), Variable(target.long()).cuda()
        optimizer.zero_grad()
        output = model(train_x)
        loss = criterion(output, train_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
    if(epoch + 1) %2 == 0:
        print(epoch+1, total_loss)

2 144.540207631886
4 77.75788171030581
6 54.96656375192106
8 43.338101097382605
10 36.97992839617655


In [7]:
test_x, test_y = Variable(test_data.test_data.float()).cuda(), Variable(test_data.test_labels.long()).cuda()
test_x = test_x.reshape(-1, 1,28,28)

In [8]:
result = torch.max(model(test_x).data, 1)[1]
accuracy = sum(test_y.cpu().numpy() == result.cpu().numpy()) / len(test_y.cpu().numpy())
accuracy

0.9782

In [14]:
model(test_x[0:10].reshape(-1, 1, 28, 28))

tensor([[   31.7112,  -131.1470,   414.8161,   451.3955,  -679.7946,   -39.0864,
         -1213.8644,  1220.6010,   135.8199,   341.7913],
        [  360.8430,   710.0635,  1241.4318,   315.2697,  -719.0433,  -320.5125,
           -35.3708,  -308.0323,   105.2062, -1000.6298],
        [ -193.2367,   618.8823,   -39.8172,   -99.9134,    36.8352,  -318.3038,
           -11.9792,   105.6962,   126.8053,  -142.6334],
        [ 1136.5857,  -422.8861,   113.5269,  -105.1617,  -288.9244,  -130.3493,
           128.3009,  -120.9538,  -192.9983,   313.8086],
        [  -83.0728,  -152.2972,   -30.8748,  -282.0391,  1076.5917,   -75.2010,
            -5.8986,    99.2855,  -146.9599,   502.3894],
        [ -211.2974,   802.5203,  -205.7221,  -135.6807,   145.8864,  -507.9320,
          -201.4762,   286.6002,   112.9949,   -36.8742],
        [ -469.6852,   176.1491,  -350.0755,  -149.7153,  1085.7234,    21.6072,
          -415.2845,   304.7545,   437.1011,   582.0380],
        [ -316.9385,   -75.